# 🦌 Logic Gate Symbol Library 🔣

`IPyElk` can serve as a jumpping off point for building different types of diagrams
based on defined shapes. This notebook demonstrates importing the logic gate symbol
library.

In [1]:
import importnb
import networkx as nx

import ipyelk
import ipyelk.nx
import ipyelk.tools.tools
from ipyelk.contrib.library import logic_gates as logic
from ipyelk.elements import layout_options as opt

# Logic Gates

https://upload.wikimedia.org/wikipedia/commons/c/cb/Circuit_elements.svg

In [2]:
gate_library = logic.Gate.make_defs()

# Simple Graph to draw the Logic Gate Symbol Library

In [3]:
def logic_library():
    # build graph
    symbols = [symbol() for symbol in logic.Gate.__subclasses__()]
    g = nx.Graph()
    tree = nx.DiGraph()

    for s in symbols:
        name = s.__class__.__name__
        g.add_node(name, **s.node(id=name).dict(exclude_none=True))

    # configure app
    app = ipyelk.from_nx(
        graph=g,
        layout={"height": "100%"},
    )
    app.view.symbols = gate_library
    return app


if __name__ == "__main__":
    app = logic_library()
    display(app)

Diagram(children=[HTML(value='<style></style>', layout=Layout(display='None')), SprottyViewer(hover=Hover(tee=…

# 1-bit memory circuit

In [4]:
sizing_opts = opt.OptionsWidget(
    options=[
        opt.NodeSizeConstraints(),
        opt.NodeLabelPlacement(horizontal="center", vertical="center"),
    ]
).value

HIGH = "on"
LOW = "off"


states = {
    ("nor 1", "a"): {
        "value": 0,
    },
    ("nor 1", "b"): {
        "value": 0,
    },
    ("nor 1", "out"): {
        "targets": [("nor 2", "b")],
        "value": 0,
    },
    ("nor 2", "a"): {
        "value": 0,
    },
    ("nor 2", "b"): {
        "value": 0,
    },
    ("nor 2", "out"): {
        "targets": [("one_b", "out"), ("nor 1", "a")],
        "value": 0,
    },
    "set": {
        "targets": [("one_b", "set")],
        "value": 0,
    },
    "reset": {
        "targets": [("one_b", "reset")],
        "value": 0,
    },
    ("one_b", "set"): {
        "targets": [("nor 1", "b")],
        "value": 0,
    },
    ("one_b", "reset"): {
        "targets": [("nor 2", "a")],
        "value": 0,
    },
    ("one_b", "out"): {
        "targets": [("output", None)],
        "value": 0,
    },
    "output": {
        "value": 0,
    },
}


def xor(a, b):
    return (a and not b) or (b and not a)


def step_state(states, set_, reset):
    nor1_a = states[("nor 1", "a")]["value"]
    nor2_a = states[("nor 2", "a")]["value"]

    nor1_b = set_
    nor2_b = reset
    nor2 = not (nor2_a or nor2_b)
    nor1 = not (nor1_a or nor1_b)

    output = nor2
    nor1_a = nor2
    nor2_a = nor1

    states[("nor 1", "a")]["value"] = nor1_a
    states[("nor 1", "b")]["value"] = nor1_b
    states[("nor 1", "out")]["value"] = nor1
    states[("nor 2", "a")]["value"] = nor2_a
    states[("nor 2", "b")]["value"] = nor2_b
    states[("nor 2", "out")]["value"] = nor2
    states["set"]["value"] = set_
    states[("one_b", "set")]["value"] = set_
    states[("one_b", "reset")]["value"] = reset
    states[("one_b", "out")]["value"] = output
    states["reset"]["value"] = reset
    states["output"]["value"] = output
    
    
def get_el_state(state, el_map):
    if isinstance(state, tuple):
        s, p = state
        return el_map[s].get_port(p)
    else:
        return el_map[state]
    
def paint(el, value):
    new, old = (HIGH, LOW) if value else (LOW, HIGH)
    el.add_class(new).remove_class(old)

def get_endpoint(u, port, el_map):
    node = el_map[u]
    if port:
        return node.get_port(port)
    return node

def update_colors(states, el_map):
    for state, data in states.items():
        value = data["value"]
        new, old = (HIGH, LOW) if value else (LOW, HIGH)

        # paint source state
        source = get_el_state(state, el_map)
        paint(source, value)

        # paint targets
        for target in data.get("targets",[]):
            paint(get_el_state(target, el_map), value)

        # paint edges
        for key, edge in el_map.edges(source=source):
            paint(edge, value)
            
        
def step(change):
    el_map = dia.view.source.index.elements
    set_ = "set" in change.new
    reset = "reset" in change.new
    step_state(states, set_, reset)
    step_state(states, set_, reset)
    update_colors(states, el_map)
    dia.view.source._notify_trait("value", old_value=None, new_value=None)    


In [5]:
from ipyelk.elements import Node, Label, Port, ElementIndex, index, Registry, EMPTY_SENTINEL

g = nx.MultiDiGraph()
tree = nx.DiGraph()

for name in ["set", 'reset', "output"]:
    g.add_node(
        Node(
            id=name,
            labels=[Label(text=name)],
            layoutOptions=sizing_opts,
        )
    )
        

one_b = Node(
    id="one_b",
    labels=[Label(text="One Bit Memory")],
    layoutOptions={
        'org.eclipse.elk.nodeSize.constraints': 'NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE',
        'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_TOP INSIDE'
    },
    ports=[
        Port(
            width="5",
            height="5",
            properties={
                "key":"set"
            },
            layoutOptions= {
                "org.eclipse.elk.port.side": "WEST",
            },
        ),
        Port(
            width="5",
            height="5",
            properties={
                "key":"reset"
            },
            layoutOptions= {
                "org.eclipse.elk.port.side": "WEST",
            },
        ),
        Port(
            width="5",
            height="5",
            properties={
                "key":"out"
            },
            layoutOptions= {
                "org.eclipse.elk.port.side": "EAST",
            },
        ),        
    ]
)        
g.add_node(one_b)

nor_gate = logic.Nor_Gate()
for gate_id in ['nor 1', "nor 2"]:
    gate = nor_gate.node(id=gate_id)
    g.add_node(gate)
    tree.add_edge(one_b, gate)
    gate.labels = [
        Label(
            text=gate_id,
            layoutOptions={
                'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_BOTTOM OUTSIDE'
            }
        )
    ]
    
with Registry():
    el_map = ElementIndex.from_els(*list(g.nodes()))
    
for state, data in states.items():
    if isinstance(state, tuple):
        u, source_port = state
    else:
        u, source_port = state, None  
    for i, target in enumerate(data.get("targets", [])):
        v, target_port = target    
        
        
        edge_data = {
            "targetPort":target_port,
            "properties":{"cssClasses": HIGH if data.get("value") else LOW},            
        }
        if source_port:
            edge_data['sourcePort']=source_port
            
        g.add_edge(
            el_map[u],
            el_map[v],
            **edge_data,
        )

dia = ipyelk.from_nx(
    graph=g,
    hierarchy=tree,
    style={
    " .on rect.elknode, .on .elkport": {
        "fill": "var(--jp-brand-color2)",
    },
    " .on .elklabel": {
        "fill": "white",
    },
    " .on, .on .elknode": {
        "stroke": "var(--jp-brand-color2)",
        "stroke-width": "4",
        "transition": "all 0.2s",
    },
    " .off, .off .elknode": {
        "transition": "all 0.2s",
    },
    },
)

dia.view.symbols = gate_library

dia        

Diagram(children=[HTML(value='<style>.styled-widget-140438077627728 .on rect.elknode, .on .elkport{fill: var(-…

In [6]:
dia.tools[1].handler()

cancelling...? False


In [7]:
dia.tools[0].observe(step,'ids')

In [8]:
from IPython.display import JSON
JSON(app.source.value.dict())

<IPython.core.display.JSON object>